In [45]:
import logging
import os
import pickle
import time
import statistics
from abc import abstractmethod

import colorsys
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import hdbscan

from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score, silhouette_samples
import statistics

from sklearn.neighbors import LocalOutlierFactor

from sklearn.svm import OneClassSVM




In [46]:
f0 = 'base.csv'
f1 = 'weakwind.csv'

base_df = pd.read_csv(f0, header=0, index_col=None)
weakwind1_df = pd.read_csv(f1, header=0, index_col=None)


df_list = [base_df, weakwind1_df]

for df in df_list:

    df['airspeedchange'] = df['airspeed']- df['airspeed'].shift(1)
    df['airspeedchange'].fillna(0, inplace=True)

base_df = base_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed',  'airspeedchange']]
clustering_weakwind1_df = weakwind1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]




In [47]:
#train the 5 models

kmeans = KMeans(init="random", n_clusters=2, n_init=10, random_state=1)
kmeans.fit(base_df)

lof = LocalOutlierFactor(n_neighbors=14, contamination="auto",novelty=True)
lof.fit(base_df)

nu = 0.1
kernel = 'rbf'
gamma = 'scale'

model = OneClassSVM(nu=nu, kernel=kernel, gamma=gamma)
model.fit(base_df)

db = DBSCAN(eps = 0.78).fit(base_df)

optics_clustering = OPTICS(min_samples = 50).fit(base_df)



In [48]:
def initialisation_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 0.04 or datapoint["climb"] < -0.02:
        rules_broken += 1
    
    if datapoint["airspeed"]!= 0.0:
        rules_broken += 1

    if datapoint["throttle"]!= 0.0:
        rules_broken += 1

    if datapoint["roll"] > -0.0003 or datapoint["roll"] < -0.0009:
        rules_broken += 1

    if datapoint["pitch"] > 0.001 or datapoint["pitch"] < 0.0009:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.0005 or datapoint["pitchspeed"] < 0.0001:
        rules_broken += 1

    if datapoint["rollspeed"] > 0.0004 or datapoint["rollspeed"] < 0.00009:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.002 or datapoint["yawspeed"] < 0.001:
        rules_broken += 1
    return rules_broken

def takeoff_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 1.6 or datapoint["climb"] < -0.2:
        rules_broken += 1
    
    if datapoint["airspeed"]>6 or datapoint["airspeed"] < 0:
        rules_broken += 1

    if datapoint["throttle"]> 80 or datapoint["throttle"] <0:
        rules_broken += 1

    if datapoint["roll"] > 0.17 or datapoint["roll"] < -0.07:
        rules_broken += 1

    if datapoint["pitch"] > 0.16 or datapoint["pitch"] < -0.19:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.1 or datapoint["pitchspeed"] < -0.15:
        rules_broken += 1

    if datapoint["rollspeed"] > 0.89 or datapoint["rollspeed"] < -0.87:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.85 or datapoint["yawspeed"] < -0.095:
        rules_broken += 1
    return rules_broken

def on_mission_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 1.6 or datapoint["climb"] < -1.1:
        rules_broken += 1
    
    if datapoint["airspeed"]>7.05 or datapoint["airspeed"] < 0.1:
        rules_broken += 1

    if datapoint["throttle"]> 100 or datapoint["throttle"] <31:
        rules_broken += 1

    if datapoint["roll"] > 0.22 or datapoint["roll"] < -0.25:
        rules_broken += 1

    if datapoint["pitch"] > 0.16 or datapoint["pitch"] < -0.3:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.2 or datapoint["pitchspeed"] < -0.22:
        rules_broken += 1

    if datapoint["rollspeed"] > 1.07 or datapoint["rollspeed"] < -1.14:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.79 or datapoint["yawspeed"] < -0.81:
        rules_broken += 1
    return rules_broken

def return_to_origin_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 0.09  or datapoint["climb"] < -0.041:
        rules_broken += 1
    
    if datapoint["airspeed"]> 6.9 or datapoint["airspeed"] < 0.04:
        rules_broken += 1

    if datapoint["throttle"]> 50 or datapoint["throttle"] < 32:
        rules_broken += 1

    if datapoint["roll"] > 0.068 or datapoint["roll"] < -0.07:
        rules_broken += 1

    if datapoint["pitch"] > 0.22 or datapoint["pitch"] < -0.062:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.088 or datapoint["pitchspeed"] < -0.087:
        rules_broken += 1

    if datapoint["rollspeed"] > 0.68 or datapoint["rollspeed"] < -0.68:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.039 or datapoint["yawspeed"] < -0.043:
        rules_broken += 1
    return rules_broken

def landing_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 0.163 or datapoint["climb"] < -1.18:
        rules_broken += 1
    
    if datapoint["airspeed"]>6.1 or datapoint["airspeed"] < 0:
        rules_broken += 1

    if datapoint["throttle"]> 38 or datapoint["throttle"] <0:
        rules_broken += 1

    if datapoint["roll"] > 0.05 or datapoint["roll"] < -0.081:
        rules_broken += 1

    if datapoint["pitch"] > 0.05 or datapoint["pitch"] < -0.07:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.04 or datapoint["pitchspeed"] < -0.05:
        rules_broken += 1

    if datapoint["rollspeed"] > 1.61 or datapoint["rollspeed"] < -1.49:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.071 or datapoint["yawspeed"] < -0.023:
        rules_broken += 1
    return rules_broken

def constant_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["GPS_status"] != 1:
        rules_broken += 1

    if datapoint["Gyro_status"] != 1:
        rules_broken += 1

    if datapoint["Accel_status"] != 1:
        rules_broken += 1

    if datapoint["Baro_status"] != 1:
        rules_broken += 1
    return rules_broken

<h3>weakwind<h3>

In [49]:
weakwind_rule_alerts = []

for i in range (len(weakwind1_df)):

    weakwind_rule_alerts.append(constant_rule_checker(weakwind1_df.iloc[i]))

    if weakwind1_df.iloc[i]["PHASE"] == "INITIALISATION":
        weakwind_rule_alerts.append(initialisation_rule_checker(weakwind1_df.iloc[i]))
    elif weakwind1_df.iloc[i]["PHASE"] == "TAKEOFF":
        weakwind_rule_alerts.append(takeoff_rule_checker(weakwind1_df.iloc[i]))
    elif weakwind1_df.iloc[i]["PHASE"] == "ON MISSION":
        weakwind_rule_alerts.append(on_mission_rule_checker(weakwind1_df.iloc[i]))
    elif weakwind1_df.iloc[i]["PHASE"] == "RETURN TO ORIGIN":
        weakwind_rule_alerts.append(return_to_origin_rule_checker(weakwind1_df.iloc[i]))
    elif weakwind1_df.iloc[i]["PHASE"] == "LANDING":
        weakwind_rule_alerts.append(landing_rule_checker(weakwind1_df.iloc[i]))

count = 0
for value in weakwind_rule_alerts:
    if value >0:
        count +=1

print("Out of the " + str(len(weakwind1_df)) + " anomalies, " + str(count) + " were found, which is " + str(count/len(weakwind1_df) * 100) + "%")



Out of the 424 anomalies, 411 were found, which is 96.93396226415094%


In [50]:
#predict for weakwind

weakwind_kmeans = []
weakwind_lof = []
weakwind_svm = []
weakwind_db = []
weakwind_optics = []

base_df2 = base_df.copy()
base_df2['cluster'] = kmeans.labels_
weakwind1_df2 = clustering_weakwind1_df.copy()
weakwind1_df2['cluster'] = kmeans.predict(clustering_weakwind1_df)

centroids = kmeans.cluster_centers_

original_distances = []
for i in range(len(base_df)):
    centroid = centroids[int(base_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((base_df.iloc[i] - centroid)**2))
    original_distances.append(distance)

original_mean = sum(original_distances)/len(original_distances)

original_std = np.std(original_distances)

threshold = original_mean + 3* original_std

for i in range(len(clustering_weakwind1_df)):
    centroid = centroids[int(weakwind1_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((clustering_weakwind1_df.iloc[i] - centroid)**2))
    if distance > threshold:
        weakwind_kmeans.append(-1)
    else:
        weakwind_kmeans.append(1)


for i in range(clustering_weakwind1_df.shape[0]):
    prediction = lof.predict(clustering_weakwind1_df.iloc[[i]])
    for value in prediction:
        weakwind_lof.append(value)

for i in range(clustering_weakwind1_df.shape[0]):
    prediction = model.predict(clustering_weakwind1_df.iloc[[i]])
    for value in prediction:
        weakwind_svm.append(value)

for value in db.fit_predict(clustering_weakwind1_df):
    weakwind_db.append(value)

for value in optics_clustering.fit_predict(clustering_weakwind1_df):
    weakwind_optics.append(value)


#get predictions for weakwind using voting

final_weakwind_predictions = []

for i in range (len(weakwind1_df)):
    predictions = [weakwind_kmeans[i], weakwind_lof[i], weakwind_svm[i], weakwind_db[i], weakwind_optics[i]]
    final_weakwind_predictions.append(max(set(predictions), key=predictions.count))
    
print("For weakwind, out of the", len(weakwind1_df), "anomalies", final_weakwind_predictions.count(-1), "were detected, which is", final_weakwind_predictions.count(-1)/len(weakwind1_df)*100,"%")


/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warni

For weakwind, out of the 424 anomalies 5 were detected, which is 1.179245283018868 %


/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


In [51]:
overall_weakwind_count = 0

for i in range(len(weakwind1_df)):
    if final_weakwind_predictions[i] <0 or weakwind_rule_alerts[i] > 0:
        overall_weakwind_count+=1

print("For weakwind, out of the", len(weakwind1_df), "anomalies", overall_weakwind_count ,"were detected, which is", overall_weakwind_count/len(weakwind1_df)*100,"%")


For weakwind, out of the 424 anomalies 207 were detected, which is 48.82075471698113 %
